In [1]:
s3_train_path = 's3://mastering-ml-aws/chapter4/training-data'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1556315839936_0012,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
ctr_df = spark.read.json(s3_train_path)


VBox()

In [3]:
ctr_df.show(5)

VBox()

+-------------+------------+-------------+-------------+------------+-------------+--------------------+-------------+------------+--------------------+---+
|           c0|          c1|           c2|           c3|          c4|           c5|                  c6|           c7|          c8|                  c9|  l|
+-------------+------------+-------------+-------------+------------+-------------+--------------------+-------------+------------+--------------------+---+
|[-1664374510]|[1292560685]| [1963151207]| [-113426919]|[1024827180]|         null|[-1841755489, -20...|  [781804810]| [677061876]|[-2054476127, 128...|  0|
| [1566608579]|[-248982458]|  [336746857]|[-1629610286]| [244157766]|         null|[-574085389, 1869...| [1065163157]| [332083152]|        [-614983515]|  0|
| [1935105702]|[1292560685]|[-1389162932]| [-113426919]|  [-8361123]|  [839761088]|                null|[-1708330775]|[1856995055]|[-1954958362, 157...|  0|
| [1718276659]| [630920017]| [1171414431]| [-113426919]| [

In [4]:
df = ctr_df.selectExpr("coalesce(c0[0],0) as f0",
                       "coalesce(c1[0],0) as f1",
                       "coalesce(c2[0],0) as f2",
                       "coalesce(c3[0],0) as f3",
                       "coalesce(c4[0],0) as f4",
                       "l as click")

VBox()

In [5]:
df.show(5)

VBox()

+-----------+----------+-----------+-----------+----------+-----+
|         f0|        f1|         f2|         f3|        f4|click|
+-----------+----------+-----------+-----------+----------+-----+
|-1664374510|1292560685| 1963151207| -113426919|1024827180|    0|
| 1566608579|-248982458|  336746857|-1629610286| 244157766|    0|
| 1935105702|1292560685|-1389162932| -113426919|  -8361123|    0|
| 1718276659| 630920017| 1171414431| -113426919| 640993460|    1|
| 1562430026| 630920017| 1639152385| 1781226914|1493440023|    0|
+-----------+----------+-----------+-----------+----------+-----+
only showing top 5 rows

In [6]:
df = df.repartition(100).cache()


VBox()

In [7]:
df.describe().show()

VBox()

+-------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|summary|                  f0|                  f1|                  f2|                 f3|                  f4|              click|
+-------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+
|  count|            12000000|            12000000|            12000000|           12000000|            12000000|           12000000|
|   mean|-6.610412663970825E7|2.5049429668800482E8|-2.915904354482062E8|5.459869260236725E7|-6.716129061083934E7|         0.18310175|
| stddev|1.2294656059145813E9| 1.287445524252859E9|1.2580392622053525E9|8.234483651283175E8|1.2429134469135067E9|0.38674993421016063|
|    min|         -2145952914|         -2125813709|         -2145112401|        -2134594413|         -2147400218|                  0|
|    max|          2146734164|          2136145316|          2

In [8]:
df.select("f0").distinct().count()

VBox()

2497

In [9]:
df.select("f1").distinct().count()

VBox()

178

In [10]:
df.select("f3").distinct().count()

VBox()

68

In [11]:
df.select("f4").distinct().count()

VBox()

17572

In [12]:
from pyspark.ml.feature import StringIndexer

string_indexer = StringIndexer(inputCol="f0", outputCol="f0_index")
string_indexer_model = string_indexer.fit(df)
ctr_df_indexed = string_indexer_model.transform(df).select('f0','f0_index')
ctr_df_indexed.show(5)

VBox()

+-----------+--------+
|         f0|f0_index|
+-----------+--------+
| -130745722|   178.0|
|-1322326169|     0.0|
| -130745722|   178.0|
|-1248885727|     8.0|
|  571589560|   877.0|
+-----------+--------+
only showing top 5 rows

In [13]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="f0_index", outputCol="f0_encoded")
encoder.transform(ctr_df_indexed).distinct().show(5)

VBox()

+-----------+--------+-------------------+
|         f0|f0_index|         f0_encoded|
+-----------+--------+-------------------+
|-1910840705|   118.0| (2496,[118],[1.0])|
|-1713169383|   242.0| (2496,[242],[1.0])|
| 1590237751|  1216.0|(2496,[1216],[1.0])|
|-1156005499|   337.0| (2496,[337],[1.0])|
| 1707433888|   388.0| (2496,[388],[1.0])|
+-----------+--------+-------------------+
only showing top 5 rows

In [14]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import ChiSqSelector

def categorical_one_hot_encoding_stages(columns):
    indexers = [StringIndexer(inputCol=column, outputCol=column + "_index", handleInvalid='keep') for column in columns]
    encoders = [OneHotEncoder(inputCol=column + "_index", outputCol=column + "_encoded") for column in columns]
    return indexers + encoders

from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

categorical_columns = ['f0','f1','f2','f3','f4']
encoded_columns = [column + '_encoded' for column in categorical_columns] 

categorical_stages = categorical_one_hot_encoding_stages(categorical_columns)
vector_assembler = VectorAssembler(inputCols=encoded_columns, outputCol="features")
selector = ChiSqSelector(numTopFeatures=100, featuresCol="features",
                         outputCol="selected_features", labelCol="click")
decision_tree = DecisionTreeClassifier(labelCol="click", featuresCol="selected_features")

pipeline = Pipeline(stages=categorical_stages + [vector_assembler, selector, decision_tree])

VBox()

In [15]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=17)


VBox()

In [16]:
train_df = train_df.repartition(100).cache()


VBox()

In [17]:
pipeline_model = pipeline.fit(train_df)

VBox()

In [18]:
print(pipeline_model.stages[-1].toDebugString)

VBox()

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_3cc3252e8007) of depth 5 with 11 nodes
  If (feature 3 in {1.0})
   Predict: 1.0
  Else (feature 3 not in {1.0})
   If (feature 21 in {1.0})
    Predict: 1.0
   Else (feature 21 not in {1.0})
    If (feature 91 in {1.0})
     Predict: 1.0
    Else (feature 91 not in {1.0})
     If (feature 27 in {1.0})
      Predict: 1.0
     Else (feature 27 not in {1.0})
      If (feature 29 in {1.0})
       Predict: 1.0
      Else (feature 29 not in {1.0})
       Predict: 0.0

In [19]:
pipeline_model

VBox()

PipelineModel_3c22c4c494ac

In [20]:
pipeline_model.save("s3://mastering-ml-aws/chapter4/pipeline-model")


VBox()

In [21]:
test_transformed = pipeline_model.transform(test_df)
test_transformed.write.parquet("s3://mastering-ml-aws/chapter4/chi-test-tranformed/")

VBox()

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="click")
evaluator.evaluate(pipeline_model.transform(test_df, {evaluator.metricName: "areaUnderROC"}))

VBox()

0.43916919878650384

In [23]:
train_transformed = pipeline_model.transform(train_df)


VBox()

In [26]:
train_transformed.select(["click"] + encoded_columns).write.parquet('s3://mastering-ml-aws/chapter4/train-trans-vector/')


VBox()

In [27]:
train_transformed.select("click", "selected_features").show(5)


VBox()

+-----+-----------------+
|click|selected_features|
+-----+-----------------+
|    0| (100,[76],[1.0])|
|    1|      (100,[],[])|
|    0| (100,[33],[1.0])|
|    0|      (100,[],[])|
|    0|  (100,[5],[1.0])|
+-----+-----------------+
only showing top 5 rows

In [35]:
def deconstruct_vector(row):
    arr = row['selected_features'].toArray()
    return tuple([row['click']] + arr.tolist())

df_for_csv = train_transformed.select("click", "selected_features") \
                .rdd.map(deconstruct_vector).toDF() 
df_for_csv.write.csv('s3://mastering-ml-aws/chapter4/train-trans-vec-csv-1/', 
                     header=False)

VBox()

In [36]:
df_for_csv = test_transformed.select("click", "selected_features").rdd.map(deconstruct_vector).toDF()
df_for_csv.write.csv('s3://mastering-ml-aws/chapter4/test-trans-vec-csv-1/', header=False)

VBox()

In [39]:
def deconstruct_vector_no_label(row):
    arr = row['selected_features'].toArray()
    return tuple(arr.tolist())

df_for_csv = test_transformed.select("selected_features") \
                             .rdd.map(deconstruct_vector_no_label) \
                             .toDF().repartition(100)
df_for_csv.write.csv('s3://mastering-ml-aws/chapter4/test-trans-vec-csv-no-label/', 
                     header=False)

VBox()

In [40]:
train_transformed.show(5)

VBox()

+-----------+----------+-----------+----------+-----------+-----+--------+--------+--------+--------+--------+------------------+---------------+----------------+--------------+-------------------+--------------------+-----------------+--------------------+--------------------+----------+
|         f0|        f1|         f2|        f3|         f4|click|f0_index|f1_index|f2_index|f3_index|f4_index|        f0_encoded|     f1_encoded|      f2_encoded|    f3_encoded|         f4_encoded|            features|selected_features|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+-----------+-----+--------+--------+--------+--------+--------+------------------+---------------+----------------+--------------+-------------------+--------------------+-----------------+--------------------+--------------------+----------+
| 1981225511| 630920017| 1639152385|1781226914| -446231114|    0|    83.0|     1.0|     2.0|     2.0|   129.0| (2492,[83],[1.0])|(

In [46]:
test_transformed.show(5)

VBox()

+-----------+---------+-----------+-----------+-----------+-----+--------+--------+--------+--------+--------+-------------------+----------------+---------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+----------+
|         f0|       f1|         f2|         f3|         f4|click|f0_index|f1_index|f2_index|f3_index|f4_index|         f0_encoded|      f1_encoded|     f2_encoded|    f3_encoded|          f4_encoded|            features|selected_features|       rawPrediction|         probability|prediction|
+-----------+---------+-----------+-----------+-----------+-----+--------+--------+--------+--------+--------+-------------------+----------------+---------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+----------+
|-2139870058|856005849|-1389162932|-1064630401|-1061368695|    0|   840.0|     2.0|     0.0|     4.0|  1260.0| (2492,[840],[

In [42]:
from pyspark.ml.classification import RandomForestClassifier
   
random_forest = RandomForestClassifier(labelCol="click",
                                       featuresCol="selected_features")
rf_model = random_forest.fit(train_transformed.select("selected_features","click"))


VBox()

In [45]:
rf_evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="click")
rf_evaluator.evaluate(rf_model.transform(test_transformed.select("selected_features","click"), 
                                         {evaluator.metricName: "areaUnderROC"}))

VBox()

0.6204587882483235